In [18]:
using PyPlot
using JuMP, Ipopt
using Combinatorics

In [19]:
include("../src/model.jl");
include("../src/diagnostics.jl");
include("../src/optimization.jl");
include("../src/plotting.jl");
include("../src/defaults.jl");

# Optimizating the Global Climate Action Policy Portfolio

In [20]:
model_og = ClimateModel();
max_deployment_og = Dict("mitigate"=>1., "remove"=>1., "geoeng"=>1., "adapt"=>1. /3.)

Dict{String,Float64} with 4 entries:
  "geoeng"   => 1.0
  "mitigate" => 1.0
  "remove"   => 1.0
  "adapt"    => 0.333333

In [21]:
controls = collect(keys(max_deployment_og));

In [22]:
control_permutations = combinations(controls) |> collect

15-element Array{Array{String,1},1}:
 ["geoeng"]                               
 ["mitigate"]                             
 ["remove"]                               
 ["adapt"]                                
 ["geoeng", "mitigate"]                   
 ["geoeng", "remove"]                     
 ["geoeng", "adapt"]                      
 ["mitigate", "remove"]                   
 ["mitigate", "adapt"]                    
 ["remove", "adapt"]                      
 ["geoeng", "mitigate", "remove"]         
 ["geoeng", "mitigate", "adapt"]          
 ["geoeng", "remove", "adapt"]            
 ["mitigate", "remove", "adapt"]          
 ["geoeng", "mitigate", "remove", "adapt"]

In [23]:
results = Dict()

for control_permutation in control_permutations    
    # Assert that some controls not be deployed
    max_deployment = copy(max_deployment_og)
    for control in controls
        if ~(control in control_permutation)
            max_deployment[control] = 0
        end
    end
    
    model = ClimateModel();
    @time m = optimize_controls!(model, max_deployment = max_deployment);
    
    results[control_permutation] = Dict("NPV" => discounted_total_control_cost(model), "status" => raw_status(m))
    
end

Solve_Succeeded
  3.414886 seconds (4.08 M allocations: 198.666 MiB, 2.23% gc time)
Solve_Succeeded
  0.032570 seconds (86.40 k allocations: 3.909 MiB)
Solve_Succeeded
  0.109533 seconds (187.69 k allocations: 5.797 MiB, 31.65% gc time)
Infeasible_Problem_Detected
  0.832702 seconds (1.11 M allocations: 20.985 MiB)
Solve_Succeeded
  0.039716 seconds (89.75 k allocations: 4.016 MiB)
Solve_Succeeded
  0.029607 seconds (69.99 k allocations: 3.661 MiB)
Solve_Succeeded
  0.045128 seconds (101.55 k allocations: 4.210 MiB)
Solve_Succeeded
  0.050715 seconds (112.51 k allocations: 4.428 MiB)
Solve_Succeeded
  0.043744 seconds (106.02 k allocations: 4.310 MiB)
Solve_Succeeded
  0.042338 seconds (108.75 k allocations: 4.355 MiB)
Solve_Succeeded
  0.040770 seconds (100.75 k allocations: 4.261 MiB)
Solve_Succeeded
  0.041223 seconds (101.21 k allocations: 4.272 MiB)
Solve_Succeeded
  0.032503 seconds (80.83 k allocations: 3.894 MiB)
Solve_Succeeded
  0.059808 seconds (116.87 k allocations: 4.564 M

In [24]:
min_value = Inf
for (index, value) in pairs(results)
    if (value["status"] == "Solve_Succeeded") & (value["NPV"] < min_value)
        min_value = value["NPV"]
    end
end

for (index, value) in pairs(results)
    if (value["status"] == "Solve_Succeeded")
        abs_value = Int64(round((value["NPV"])))
        rel_value = Int64(round((value["NPV"]/min_value - 1.)*100))
        print("$index, $abs_value, $rel_value%\n")
    else
        print("$index has no solution.\n")
    end
end


["geoeng", "mitigate", "remove"], 56, 4%
["remove"], 328, 507%
["mitigate", "remove"], 61, 14%
["mitigate"], 84, 56%
["geoeng", "mitigate", "remove", "adapt"], 54, 0%
["geoeng", "adapt"], 302, 459%
["geoeng", "mitigate", "adapt"], 69, 28%
["mitigate", "remove", "adapt"], 59, 10%
["geoeng"], 420, 678%
["geoeng", "remove", "adapt"], 149, 177%
["mitigate", "adapt"], 79, 46%
["geoeng", "mitigate"], 73, 36%
["remove", "adapt"], 252, 366%
["geoeng", "remove"], 189, 250%
["adapt"] has no solution.
